In [ ]:
# Importação de bibliotecas
import pandas as pd  # Biblioteca para manipulação de dados
# Baixa o recurso 'stopwords' se não estiver disponível
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords  # Biblioteca para lidar com palavras de parada (stopwords), que são palavras comuns em um idioma
from nltk.stem.porter import PorterStemmer  # Biblioteca para aplicar stemming, reduzindo palavras às suas raízes morfológicas
import re  # Biblioteca para expressões regulares, usada para manipulação e busca de padrões em strings
from tensorflow.keras.models import Sequential  # Biblioteca para criar modelos de redes neurais sequenciais (profundas)
from tensorflow.keras.layers import Dense  # Biblioteca para criar camadas densas em redes neurais
from sklearn import preprocessing  # Biblioteca para pré-processamento de dados, como padronização e normalização
from sklearn.feature_extraction.text import CountVectorizer  # Biblioteca para converter texto em vetores de contagem de palavras
from sklearn.model_selection import train_test_split  # Biblioteca para dividir dados em conjuntos de treino e teste
import pickle  # Biblioteca para salvar e carregar objetos em formato serializado
import numpy as np # Biblioteca para cálculos numéricos

# Carregamento dos dados de treinamento, validação e teste
treino = pd.read_table('treino.txt', delimiter=';', header=None)
validacao = pd.read_table('validacao.txt', delimiter=';', header=None)
teste = pd.read_table('teste.txt', delimiter=';', header=None)

In [ ]:
# Concatenando os conjuntos de dados para facilitar a manipulação
dados = pd.concat([treino, validacao, teste])

# Definindo os nomes das colunas
dados.columns = ["texto", "emocao"]

In [ ]:
# Verificando o número de linhas do conjunto de dados
print('Número de linhas:', dados.shape[0])

# Verificando a quantidade de valores nulos nas linhas
print('Valores nulos:', dados.isna().any(axis=1).sum())

Número de linhas: 20000
Valores nulos: 0


In [ ]:
# Pré-processamento do texto
ps = PorterStemmer()  # Inicializa o stemmer (PorterStemmer) para aplicar stemming

def preprocessar(linha):
    # Remove caracteres não alfabéticos
    revisão = re.sub('[^a-zA-Z]', ' ', linha)
    # Converte o texto para minúsculas
    revisão = revisão.lower()
    # Divide o texto em lista de palavras
    revisão = revisão.split()
    # Aplica stemming e remove palavras de parada (stopwords)
    revisão = [ps.stem(palavra) for palavra in revisão if palavra not in stopwords.words('english')]
    # Junta a lista de palavras em uma frase
    return " ".join(revisão)

# Aplica o pré-processamento a todos os textos
dados['texto'] = dados['texto'].apply(lambda x: preprocessar(x))

In [ ]:
# Codificação dos rótulos para valores numéricos
label_encoder = preprocessing.LabelEncoder()
dados['emocao_numerica'] = label_encoder.fit_transform(dados['emocao'])

# Criação do modelo Bag of Words usando CountVectorizer para converter texto em dados numéricos
cv = CountVectorizer(max_features=5000, ngram_range=(1,3))
data_cv = cv.fit_transform(dados['texto']).toarray()

# Divisão dos dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(data_cv, dados['emocao_numerica'], test_size=0.25, random_state=42)

In [ ]:
# Inicializando o modelo sequencial
model = Sequential()
# Adiciona uma camada densa com 12 neurônios e função de ativação 'relu'
model.add(Dense(12, input_shape=(X_train.shape[1],), activation='relu'))
# Adiciona uma segunda camada densa com 8 neurônios e função de ativação 'relu'
model.add(Dense(8, activation='relu'))
# Adiciona uma camada de saída com 6 neurônios (número de classes) e função de ativação 'softmax'
model.add(Dense(6, activation='softmax'))

# Compila o modelo com função de perda de entropia cruzada categórica e otimizador 'adam'
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Treina o modelo com os dados de treinamento
model.fit(X_train, y_train, epochs=10, batch_size=10)

# Avalia a precisão do modelo nos dados de treinamento
_, accuracy = model.evaluate(X_train, y_train)
print(f'Precisão nos dados de treinamento: {accuracy * 100:.2f}%')

# Avalia a precisão do modelo nos dados de teste
_, accuracy = model.evaluate(X_test, y_test)
print(f'Precisão nos dados de teste: {accuracy * 100:.2f}%')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.5085 - loss: 1.3203
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9016 - loss: 0.3437
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9573 - loss: 0.1470
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9779 - loss: 0.0793
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9840 - loss: 0.0502
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9887 - loss: 0.0356
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9918 - loss: 0.0269
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9947 - loss: 0.0207
Epoch 9/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9951 - loss: 0.0158
Epoch 10/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9962 - loss: 0.0124
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9967 - loss: 0.0098
Precisão nos dados de treinamento: 99.69%
1

In [ ]:
# Função para prever o rótulo de um novo texto
texto = 'I am devasted about the death of my cat'  # Texto a ser previsto
texto = preprocessar(texto)  # Pré-processamento do texto
array = cv.transform([texto]).toarray()  # Converte o texto para vetor de características
pred = model.predict(array)  # Faz a previsão com o modelo
a = np.argmax(pred, axis=1)  # Encontra a classe com a maior probabilidade
# Exibe o rótulo correspondente ao índice previsto
print(label_encoder.inverse_transform(a)[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
sadness


In [ ]:
# Salva o modelo em um arquivo .h5
model.save('modelo_texto_emocao.h5')

# Salva o codificador de rótulos (label encoder) em um arquivo pickle
pickle.dump(label_encoder, open('encoder.pkl', 'wb'))

# Salva o CountVectorizer em um arquivo pickle
pickle.dump(cv, open('CountVectorizer.pkl', 'wb'))